In [40]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor, export_text
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, root_mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

import warnings
warnings.filterwarnings('ignore')



In [10]:
# Download data
!wget https://github.com/alexeygrigorev/datasets/raw/refs/heads/master/jamb_exam_results.csv


--2024-10-30 10:10:43--  https://github.com/alexeygrigorev/datasets/raw/refs/heads/master/jamb_exam_results.csv
Resolving github.com (github.com)... 20.26.156.215
Connecting to github.com (github.com)|20.26.156.215|:443... connected.
HTTP request sent, awaiting response... 

302 Found
Location: https://raw.githubusercontent.com/alexeygrigorev/datasets/refs/heads/master/jamb_exam_results.csv [following]
--2024-10-30 10:10:43--  https://raw.githubusercontent.com/alexeygrigorev/datasets/refs/heads/master/jamb_exam_results.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 391501 (382K) [text/plain]
Saving to: ‘jamb_exam_results.csv’

jamb_exam_results.c 100%[===================>] 382.33K  --.-KB/s    in 0.003s  

2024-10-30 10:10:43 (127 MB/s) - ‘jamb_exam_results.csv’ saved [391501/391501]



In [11]:
# Load data into dataframe and cleanup column names
df = pd.read_csv('jamb_exam_results.csv')
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [12]:
# Remove student_id column
df = df.drop(columns=['student_id'])

# Fill NaN
df = df.fillna(0)



In [14]:
# Split daya in validation, train and test
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)  # 0.25 * 0.8 = 0.2


In [15]:
# Target variable for train, vaidation and test
y_train = df_train.jamb_score.values
y_val = df_val.jamb_score.values
y_test = df_test.jamb_score.values

# Drop target from the feature dataframes
df_train = df_train.drop(columns=['jamb_score'])
df_val = df_val.drop(columns=['jamb_score'])
df_test = df_test.drop(columns=['jamb_score'])


In [16]:
# Convert DataFrames to Matrices
 
dv = DictVectorizer(sparse=True)

X_train = dv.fit_transform(df_train.to_dict(orient="records"))
X_val = dv.transform(df_val.to_dict(orient="records"))
X_test = dv.transform(df_test.to_dict(orient="records"))

## Question 1
Let's train a decision tree regressor to predict the jamb_score variable.

        Train a model with max_depth=1.
    
Which feature is used for splitting the data?

In [17]:
# Train Decision Tree Regressoor model
dt = DecisionTreeRegressor(max_depth=1, random_state=1)
dt.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=1, random_state=1)

In [25]:
# Feature used for splitting the data
print(export_text(dt, feature_names= dv.get_feature_names_out()))

|--- study_hours_per_week <= 18.50
|   |--- value: [155.24]
|--- study_hours_per_week >  18.50
|   |--- value: [188.59]



## Question 2
    
    Train a random forest model with these parameters:

        n_estimators=10
        random_state=1
        n_jobs=-1 (optional - to make training faster)

What's the RMSE of this model on validation?

In [32]:
# Initialize Random Forest Regressor model with hyperparameters
rf = RandomForestRegressor(n_estimators=10,
                            random_state=1,
                            n_jobs=-1)

# Train the model on the training set
rf.fit(X_train, y_train)

# Make predictions on the validation set
y_pred = rf.predict(X_val)

# Calculate RMSE
rmse = root_mean_squared_error(y_pred, y_val)
print(f"RMSE on validation set: {rmse:.2f}")


RMSE on validation set: 42.14


## Question 3

Now let's experiment with the n_estimators parameter

    Try different values of this parameter from 10 to 200 with step 10.
    Set random_state to 1.
    Evaluate the model on the validation dataset.

After which value of n_estimators does RMSE stop improving? *Consider 3 decimal places for calculating the answer.*

In [36]:
# Store results for comparison
rmse_results = []

# Loop through n_estimators values
for n in range(10, 201, 10):
    rf = RandomForestRegressor(n_estimators=n, random_state=1)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    rmse_results.append((n, rmse))
    print(f"n_estimators: {n}, RMSE: {rmse:.3f}")

# Find the minimum RMSE and when it stops improving
min_rmse = min(rmse_results, key=lambda x: x[1])
print(f"Minimum RMSE: {min_rmse[1]:.3f} with n_estimators: {min_rmse[0]}")


n_estimators: 10, RMSE: 42.137
n_estimators: 20, RMSE: 41.461
n_estimators: 30, RMSE: 41.106
n_estimators: 40, RMSE: 40.917
n_estimators: 50, RMSE: 40.852
n_estimators: 60, RMSE: 40.784
n_estimators: 70, RMSE: 40.677
n_estimators: 80, RMSE: 40.539
n_estimators: 90, RMSE: 40.504
n_estimators: 100, RMSE: 40.517
n_estimators: 110, RMSE: 40.593
n_estimators: 120, RMSE: 40.625
n_estimators: 130, RMSE: 40.651
n_estimators: 140, RMSE: 40.595
n_estimators: 150, RMSE: 40.597
n_estimators: 160, RMSE: 40.604
n_estimators: 170, RMSE: 40.628
n_estimators: 180, RMSE: 40.641
n_estimators: 190, RMSE: 40.631
n_estimators: 200, RMSE: 40.601
Minimum RMSE: 40.504 with n_estimators: 90


## Question 4

Let's select the best max_depth:

    Try different values of max_depth: [10, 15, 20, 25]
    For each of these values, try different values of n_estimators from 10 till 200 (with step 10)
    calculate the mean RMSE
    Fix the random seed: random_state=1

What's the best max_depth, using the mean RMSE?


In [48]:
# Initialize a dictionary to store the mean RMSE for each max_depth
mean_rmse_per_depth = {}

# Loop through each max_depth value
for depth in [10, 15, 20, 25]:
    rmse_values = []
    
    # Loop through n_estimators from 10 to 200 in steps of 10
    for n in range(10, 201, 10):
        # Initialize and train the model
        rf = RandomForestRegressor(max_depth=depth, n_estimators=n, random_state=1)
        rf.fit(X_train, y_train)
        
        # Predict on the validation set
        y_pred = rf.predict(X_val)
        
        # Calculate RMSE and append it to the rmse_values list
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        rmse_values.append(rmse)
    
    # Calculate the mean RMSE for the current max_depth
    mean_rmse_per_depth[depth] = np.mean(rmse_values)

# Find the max_depth with the minimum mean RMSE
best_depth = min(mean_rmse_per_depth, key=mean_rmse_per_depth.get)

print(f"Best max_depth: {best_depth} with Mean RMSE: {mean_rmse_per_depth[best_depth]:.3f}")


Best max_depth: 10 with Mean RMSE: 40.392


## Question 5
We can extract feature importance information from tree-based models.

    At each step of the decision tree learning algorithm, it finds the best split. When doing it, we can calculate "gain" - the reduction in impurity before and after the split. This gain is quite useful in understanding what are the important features for tree-based models.

    In Scikit-Learn, tree-based models contain this information in the feature_importances_ field.

For this homework question, we'll find the most important feature:

    Train the model with these parameters:
        n_estimators=10,
        max_depth=20,
        random_state=1,
        n_jobs=-1 (optional)
    Get the feature importance information from this model
What's the most important feature (among these 4)

In [39]:
# Initialize and train the model
rf = RandomForestRegressor(n_estimators=10, max_depth=20, random_state=1, n_jobs=-1)
rf.fit(X_train, y_train)

# Get feature importances
importances = rf.feature_importances_

# Create a DataFrame to show feature names and their importance
feature_importances = pd.DataFrame({'feature': dv.get_feature_names_out(), 'importance': importances})

# Sort features by importance
feature_importances = feature_importances.sort_values(by='importance', ascending=False)

# Display the most important feature
print(feature_importances.head(1))


                 feature  importance
27  study_hours_per_week    0.248354


## Question 6
Now let's train an XGBoost model! For this question, we'll tune the eta parameter:

    Install XGBoost
    Create DMatrix for train and validation
    Create a watchlist
    Train a model with these parameters for 100 rounds:
        xgb_params = {
            'eta': 0.3, 
            'max_depth': 6,
            'min_child_weight': 1,
            
            'objective': 'reg:squarederror',
            'nthread': 8,
            
            'seed': 1,
            'verbosity': 1,
        }

    Now change eta from 0.3 to 0.1.
Which eta leads to the best RMSE score on the validation dataset?


In [43]:
# Convert feautures and target to DMatric format for Xgboost model
dtrain = xgb.DMatrix(X_train, label=y_train)
dvalid = xgb.DMatrix(X_val, label=y_val)

# Create watchlist
watchlist = [(dtrain, 'train'), (dvalid, 'valid')]

# Define parameters
xgb_params = {
    'eta': 0.3,  # Learning rate
    'max_depth': 6,
    'min_child_weight': 1,
    'objective': 'reg:squarederror',  # Regression task
    'nthread': 8,  # Use all available cores
    'seed': 1,
    'verbosity': 1,
}

# Train the model
model = xgb.train(xgb_params, dtrain, num_boost_round=100, evals=watchlist)


[0]	train-rmse:42.69384	valid-rmse:44.89114
[1]	train-rmse:39.83326	valid-rmse:43.07010
[2]	train-rmse:37.94542	valid-rmse:42.00332
[3]	train-rmse:36.56125	valid-rmse:41.46452
[4]	train-rmse:35.44252	valid-rmse:40.88896
[5]	train-rmse:34.57756	valid-rmse:40.69096
[6]	train-rmse:33.84230	valid-rmse:40.59315
[7]	train-rmse:33.25929	valid-rmse:40.47993
[8]	train-rmse:32.79415	valid-rmse:40.45326
[9]	train-rmse:32.16019	valid-rmse:40.43929
[10]	train-rmse:31.63404	valid-rmse:40.48319
[11]	train-rmse:31.17673	valid-rmse:40.68201
[12]	train-rmse:30.87313	valid-rmse:40.63522
[13]	train-rmse:30.30310	valid-rmse:40.70983
[14]	train-rmse:30.00098	valid-rmse:40.78133
[15]	train-rmse:29.41497	valid-rmse:40.86107
[16]	train-rmse:29.25816	valid-rmse:40.96580
[17]	train-rmse:28.59378	valid-rmse:41.12190
[18]	train-rmse:28.27990	valid-rmse:41.14360
[19]	train-rmse:27.94572	valid-rmse:41.22835
[20]	train-rmse:27.49658	valid-rmse:41.27921
[21]	train-rmse:27.25449	valid-rmse:41.32427
[22]	train-rmse:27.0

In [44]:
# Change eta to 0.1
xgb_params['eta'] = 0.1

# Train the model again with the new parameter
model_eta_0_1 = xgb.train(xgb_params, dtrain, num_boost_round=100, evals=watchlist)


[0]	train-rmse:45.49999	valid-rmse:47.00533
[1]	train-rmse:44.12948	valid-rmse:45.92344
[2]	train-rmse:42.94858	valid-rmse:44.98366
[3]	train-rmse:41.90896	valid-rmse:44.25755
[4]	train-rmse:40.96728	valid-rmse:43.57339
[5]	train-rmse:40.17514	valid-rmse:43.11181
[6]	train-rmse:39.40436	valid-rmse:42.61054
[7]	train-rmse:38.71199	valid-rmse:42.18883
[8]	train-rmse:38.08081	valid-rmse:41.86754
[9]	train-rmse:37.57559	valid-rmse:41.64338
[10]	train-rmse:37.07490	valid-rmse:41.39235
[11]	train-rmse:36.58709	valid-rmse:41.14265
[12]	train-rmse:36.14574	valid-rmse:40.95201
[13]	train-rmse:35.76896	valid-rmse:40.81778
[14]	train-rmse:35.40802	valid-rmse:40.75008
[15]	train-rmse:35.08521	valid-rmse:40.61341
[16]	train-rmse:34.74187	valid-rmse:40.51800
[17]	train-rmse:34.43969	valid-rmse:40.41659
[18]	train-rmse:34.16132	valid-rmse:40.33546
[19]	train-rmse:33.91836	valid-rmse:40.25632
[20]	train-rmse:33.67389	valid-rmse:40.25010
[21]	train-rmse:33.45853	valid-rmse:40.19826
[22]	train-rmse:33.2

In [47]:
# Predictions with eta = 0.3
y_pred_eta_0_3 = model.predict(dvalid)
rmse_eta_0_3 = root_mean_squared_error(y_val, y_pred_eta_0_3)

# Predictions with eta = 0.1
y_pred_eta_0_1 = model_eta_0_1.predict(dvalid)
rmse_eta_0_1 = root_mean_squared_error(y_val, y_pred_eta_0_1)

print(f"RMSE with eta=0.3: {rmse_eta_0_3:.3f}")
print(f"RMSE with eta=0.1: {rmse_eta_0_1:.3f}")


RMSE with eta=0.3: 43.419
RMSE with eta=0.1: 41.050
